In [1]:
import numpy as np
import matplotlib.pyplot as plt

project = 'newfault'
traindate = '2024-10-01'
testdate = '2024-10-02'
traindate_path = '/home/sdybing/gnss-picker/cnn_models_outputs/' + project + '_fq_train/models/traindate_' + traindate + '/'
test_outputs_path = traindate_path + 'data/'
figure_save_dir = traindate_path + 'figures/'

fqtest_data = np.load(test_outputs_path + testdate + '_fqtest_norm_data.npy')
fqtest_metadata = np.load(test_outputs_path + testdate + '_fqtest_metadata.npy')
fqtest_target = np.load(test_outputs_path + testdate + '_fqtest_target.npy')
fqtest_predictions = np.load(test_outputs_path + testdate + '_fqtest_predictions.npy')

num_fqtest = len(fqtest_predictions)
best_thresh = 0.135 # From code 2

zeros = np.zeros((fqtest_predictions.shape[0],1))
analysis_array = np.c_[fqtest_metadata, zeros] # Adds a column of zeros to the metadata array to initialize


In [2]:
results = np.load(test_outputs_path + 'fakequakes_testing/fqtest_metadata_withresults.npy')

In [35]:
results[:,3]

array(['false neg', 'true pos', 'true neg', ..., 'false neg', 'true pos',
       'false neg'], dtype='<U32')

In [5]:
tp = 0
tn = 0
fp = 0
fn = 0

for i in range(len(results)):
    
    result = results[i,3]
#     print(result)
    if result == 'true pos':
        tp += 1
    elif result == 'true neg':
        tn += 1
    elif result == 'false pos':
        fp += 1
    elif result == 'false neg':
        fn += 1
    else:
        print('Error')

In [62]:
# Define confusion matrix values
# Format: [[TN, FP], [FN, TP]]
conf_matrix = np.array([[tn, fp],   # TN, FP
                        [fn, tp]])  # FN, TP
conf_matrix_names = np.array([['TN', 'FP'],   # TN, FP
                        ['FN', 'TP']])

# Create a figure and a single subplot
fig, ax = plt.subplots(dpi = 400)

# Plot the matrix with transposed axes (True on x-axis, Predicted on y-axis)
cax = ax.imshow(conf_matrix.T, cmap='Blues')

# Add colorbar for scale
plt.colorbar(cax, label = 'Count')

# Define class names
classes = ['Noise', 'Earthquake']

# Add labels for x and y axes (swapped now)
ax.set_xticks(np.arange(len(classes)))
ax.set_yticks(np.arange(len(classes)))
ax.set_xticklabels(classes, fontsize = 12)
ax.set_yticklabels(classes, fontsize = 12, rotation = 90, va = 'center')

# Add axis labels (swapped)
plt.xlabel('Actual label', fontsize = 14)
plt.ylabel('Predicted label', fontsize = 14)

# Annotate each cell with the TP, TN, FP, FN values
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        if i == 0 and j == 0:
            color = 'white'
        elif i == 1 and j == 0 or i == 0 and j == 1:
            color = 'tomato'
        else:
            color = 'black'
        ax.text(i, j-0.13, f'{conf_matrix_names[i, j]}', ha='center', va='center', color=color, fontsize=18)
        ax.text(i, j, f'{round((conf_matrix[i, j]/len(results))*100,1)}%', ha='center', va='center', color=color, fontsize=14)
        ax.text(i, j+0.13, f'{conf_matrix[i, j]}', ha='center', va='center', color=color, fontsize=10)
        

# Display the plot
# plt.show()

plt.savefig('/home/sdybing/gnss-picker/manuscript_figures/fq_testdata_confusion_matrix.jpg', format = 'JPG')
plt.close();

In [7]:
print(tp)
print(tn)
print(fp)
print(fn)

10476
44257
1613
35394
